In [1]:
import pandas as pd
import papermill as pm
from datetime import datetime, time
import pickle
from utils.utils_dates import map_month_folder
import os

In [2]:
from config.config_loader import Config

config = Config()

In [3]:
# Parameters
nb_verif = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/notebooks/verification_project/5B_Ver_Notebook.ipynb"
issue_year = 2025
issue_month = 6
cliente = "Kem One"
alias = "KEM ONE"
tarifa = "6.2 TD"
cups = "ES0021000003225511XB"
path_load_parquet = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/data/processed/elec/elec_load.parquet"
path_contract_matrix = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/data/customers/elec/Kem One/contract_matrix_ES0021000003225511XB.pkl"
path_coberturas_df = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/data/customers/elec/Kem One/df_coberturas_1.pkl"
path_master_matrix = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/data/customers/elec/Kem One/master_matrix_ES0021000003225511XB.pkl"
db_elec_path = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/data/processed/elec/facturas_elec.db"
db_elec_manual_path = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/data/processed/elec/facturas_elec_manual.pkl"
template_path = "/Users/mikelperez/Python_projects/0_Python_projects_v2/0_Python_projects_v2/data/verification_project/elec/templates/Template 2024.xlsx"


In [4]:
# Parameters
nb_verif = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/notebooks/verification_project/5B_Ver_Notebook.ipynb"
issue_year = 2025
issue_month = 6
cliente = "Generic"
alias = "TRIAL"
tarifa = "6.1 TD"
cups = "TRIAL"
path_load_parquet = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/elec_load.parquet"
path_contract_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/contract_matrix_TRIAL.pkl"
path_coberturas_df = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/df_coberturas_1.pkl"
path_master_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/customers/elec/Generic/master_matrix_TRIAL.pkl"
db_elec_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec.db"
db_elec_manual_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/facturas_elec_manual.pkl"
template_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/templates/Template 2024.xlsx"


In [5]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

facturas_df_sql = pd.read_sql(('SELECT * FROM facturas'), con=engine);

facturas_df_manual = pd.read_pickle(db_elec_manual_path)

# if not facturas_df[facturas_df['cups'] == cups].empty:

facturas_df = pd.concat([facturas_df_sql, facturas_df_manual], ignore_index=True)
for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
    facturas_df[column] = pd.to_datetime(facturas_df[column])

2025-08-06 12:02:39,738 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2025-08-06 12:02:39,738 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")


2025-08-06 12:02:39,738 INFO sqlalchemy.engine.Engine [raw sql] ()


2025-08-06 12:02:39,739 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")


2025-08-06 12:02:39,739 INFO sqlalchemy.engine.Engine [raw sql] ()


2025-08-06 12:02:39,739 INFO sqlalchemy.engine.Engine SELECT * FROM facturas


2025-08-06 12:02:39,739 INFO sqlalchemy.engine.Engine [raw sql] ()


2025-08-06 12:02:39,752 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
# numbers = [
#     9500049892, 9500049870, 9500048959, 9500048960, 9500049885, 9500049894, 
#     9500049702, 9500049875, 9500048938, 9500048939, 9500048941, 9500048942, 
#     9500048943, 9500048936, 9500048940, 9500048937, 9500048944, 9500049891
# ]
# for i in range (len(numbers)):

#     numbers[i] = str(numbers[i])

# numbers

In [7]:
# We first look for issued invoices in our target month-year

df_filtered_issued = facturas_df[
                        (facturas_df['cups'] == cups) &
                        (facturas_df['issue_date'].dt.month == issue_month) &
                        (facturas_df['issue_date'].dt.year == issue_year)
                        ]

# Now we look for all the invoices that have the start and end invoice dates of the invoices issued
# on our target month-year

df_issued_compile = facturas_df[
                        (facturas_df['cups'] == cups) &
                        (facturas_df['inicio_periodo'].isin(df_filtered_issued['inicio_periodo'])) &
                        (facturas_df['fin_periodo'].isin(df_filtered_issued['fin_periodo']))
                        # eliminar el filtro de debajo, es solo para Kem One
                        # &
                        # (facturas_df['num_factura'].str.contains('ELE'))
                        ]

# df_issued_compile = facturas_df[
#                         (facturas_df['num_factura'].isin(numbers)) &
#                         (facturas_df['cups'] == cups)
#                         ]

df_issued_compile = df_issued_compile.copy()
df_issued_compile.drop(['issue_date', 'year', 'month'], axis=1, inplace=True)

# Group by 'cups', 'inicio_periodo', 'fin_periodo' while keeping invoice references
ds = df_issued_compile.groupby(['cups', 'inicio_periodo', 'fin_periodo']).agg({
    'num_factura': lambda x: ', '.join(x.astype(str)),  # Concatenate all invoice references
    **{col: 'sum' for col in df_issued_compile.columns if col not in ['cups', 'inicio_periodo', 'fin_periodo', 'num_factura']}
})


df = pd.DataFrame(ds)
df.reset_index(drop=False, inplace=True)
# df_issued_compile
df
# Check how to include one row of the dataframe as parameter using papermill

,cups,inicio_periodo,fin_periodo,num_factura,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,total_bruto_ie_iva,...,energia_p3,energia_p4,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6


In [8]:
if not df.empty:

    for index, row in df.iterrows():

        start_date = row['inicio_periodo'].date()
        end_date = row['fin_periodo'].date()
        start_date = datetime.combine(start_date, time(0,0,0))
        end_date = datetime.combine(end_date, time(23,0,0))
        start_path = start_date.strftime("%Y%m%d")
        end_path = end_date.strftime("%Y%m%d")
        invoice_refs = row['num_factura']
        invoice_dict = row.to_dict()
        month_clean = map_month_folder(issue_month)

        invoice_path_pickle = \
            config.get_path("outputs.verification_project.verif.cliente.invoice_path_pickle", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        output_verif_excel = \
            config.get_path("outputs.verification_project.verif.cliente.output_verif_excel", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        output_verif_pdf = \
            config.get_path("outputs.verification_project.verif.cliente.output_verif_pdf", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        output_verif_nb = \
            config.get_path("outputs.verification_project.verif.cliente.output_verif_nb", 
                            customer_id = cliente,
                            start_date=start_path, 
                            end_date=end_path,
                            alias=alias,
                            year=issue_year,
                            month=month_clean
                            )
        
        for output_path in [invoice_path_pickle, output_verif_excel, output_verif_nb, output_verif_pdf]:

            if not os.path.exists(os.path.dirname(output_path)):

                os.makedirs(os.path.dirname(output_path))
                print(output_path)

        with open(invoice_path_pickle, "wb") as pickle_file:
            pickle.dump(invoice_dict, pickle_file) 

        pm.execute_notebook(
            nb_verif, output_verif_nb,##############
            parameters=dict(
                start_date = start_date.isoformat(),
                end_date = end_date.isoformat(),
                cliente=cliente,
                alias = alias,
                tarifa = tarifa,
                cups = cups,
                path_load_parquet = path_load_parquet,
                path_coberturas_df = path_coberturas_df,
                path_master_matrix = path_master_matrix,
                path_contract_matrix = path_contract_matrix,
                db_elec_path = db_elec_path,
                template_path = template_path,
                invoice_path_pickle = invoice_path_pickle,
                output_verif_excel =  output_verif_excel,
                output_verif_pdf = output_verif_pdf
            )
        )

    print(start_date)

    print(output_verif_pdf)

else:

    print('No inovoices to verify')

No inovoices to verify
